In [70]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [71]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [72]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [73]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [74]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [75]:
TEST_CASE_GENERATOR="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [76]:
def sanitize_string(data_str):
    # Replace single quotes with double quotes
    data_str = data_str.replace("'", '"')
    
    # Escape double quotes inside the values
    in_string = False
    sanitized_str = ""
    for i in range(len(data_str)):
        if data_str[i] == '"':
            if in_string and (i == 0 or data_str[i-1] != '\\'):
                sanitized_str += '\\"'
            else:
                sanitized_str += '"'
            in_string = not in_string if data_str[i] == '"' and (i == 0 or data_str[i-1] != '\\') else in_string
        else:
            sanitized_str += data_str[i]
    return sanitized_str

In [77]:
use_case_directory_path = r"D:\Dissertation-GPT\dataset\SpecificationData\code/thu/8.1/gen tc.ipynb"
save_path = r"D:\Dissertation-GPT\evaluate\baseline 3 1.8\Book"

In [78]:
usecase_scenario = {
    "Add a new author": [
        "1. Successful Author Creation",
        "2. Author Creation with Missing Name",
        "3. Author Creation with Missing Bio",
        "4. Author Creation with Invalid Data",
        "5. Author Creation with Duplicate Name",
        "8. Navigation Away During Author Creation (To Books)",
        "9. Navigation Away During Author Creation (To Genres)",
        "10. Navigation Away During Author Creation (To Authors)",
        "11. Navigation Away During Author Creation (To Home)",
        "12. Navigation Away During Author Creation (To API)"
    ],
    "Add a new book": [
        "1. Successful Book Creation",
        "2. Book Creation with Missing Title",
        "3. Book Creation with Missing Author",
        "4. Book Creation with Missing Genre",
        "5. Book Creation with Missing Description",
        "6. Book Creation with Missing Publication Date",
        "7. Book Creation with Invalid Publication Date Format",
        "8. Book Creation with Duplicate Title and Author",
        "10. Cancel Book Creation Before Submitting",
        "11. Navigate Away Using Menu Before Submitting"
    ],
    "Add a new genre": [
        "1. Successful Genre Creation",
        "2. Genre Creation with Missing Name",
        "3. Genre Creation with Missing Description",
        "4. Genre Creation with Duplicate Name",
        "8. Cancel Genre Creation Before Submitting",
        "10. Navigation Away During Genre Creation Without Saving",
        "12. Genre Creation with Script Injection in Name",
        "13. Genre Creation with Script Injection in Description",
        "14. Genre Creation with HTML Tags in Name",
        "15. Genre Creation with HTML Tags in Description"
    ],
    "Display author details": [
        "1. Display Author Details - Valid Author Selection",
        "2. Display Author Details - Author With No Biography",
        "3. Display Author Details - Author With Extensive Biography",
        "4. Display Author Details - Nonexistent Author Selection",
        "5. Display Author Details - Author With Special Characters in Name",
        "8. Display Author Details - Access Through Direct URL",
        "10. Display Author Details - Navigation Back to Author List"
    ],
    "Display book details": [
        "1. DisplayBookDetails_ValidBookSelection",
        "3. DisplayBookDetails_BookWithNoAuthor",
        "4. DisplayBookDetails_BookWithNoGenre",
        "5. DisplayBookDetails_BookWithNoDescription",
        "6. DisplayBookDetails_BookWithFuturePublicationDate",
        "7. DisplayBookDetails_BookWithPastPublicationDate",
        "9. DisplayBookDetails_BookWithLongDescription",
        "10. DisplayBookDetails_BookWithSpecialCharactersInTitle",
        "11. DisplayBookDetails_BookWithSpecialCharactersInAuthorName",
        "12. DisplayBookDetails_BookWithLargeID"
    ],
    "Display genre details": [
        "1. DisplayGenreDetails_Successful",
        "6. DisplayGenreDetails_SpecialCharactersInGenreName",
        "7. DisplayGenreDetails_LongTextInDescription",
        "9. DisplayGenreDetails_NumericValuesInGenreName",
        "10. DisplayGenreDetails_GenreWithHTMLTagsInDescription"
    ],
    "Display the list of authors": [
        "1. DisplayFirstPageOfAuthors",
        "2. DisplaySecondPageOfAuthorsUsingNextButton",
        "3. DisplaySpecificPageOfAuthorsUsingPageNumber",
        "4. DisplayLastPageOfAuthors",
        "5. DisplayEmptyAuthorsList",
        "6. DisplayAuthorsListWithMaximumRecordsPerPage",
        "7. DisplayAuthorsListWithSingleAuthor",
        "8. DisplayAuthorsListWithAuthorsHavingLongBios",
        "9. DisplayAuthorsListWithSpecialCharactersInNameOrBio",
        "10. NavigateBackToPreviousPageOfAuthorsFromSubsequentPage"
    ],
    "Display the list of books": [
        "1. DisplayInitialBooksList - Verify that the initial list of books is displayed correctly upon selecting the \"Books\" item.",
        "2. PaginationNextButtonFunctionality - Verify that clicking the \"Next\" button displays the next 10 records of books.",
        "3. PaginationPageNumberFunctionality - Verify that clicking a specific page number displays the corresponding 10 records of books.",
        "5. DisplayBooksListWithinDateRange - Verify that the list of books is correctly filtered and displayed when a valid date range is provided and the refresh button is clicked.",
        "6. DisplayBooksListOutsideDateRange - Verify the behavior when the provided date range does not match any book's publication date.",
        "7. DisplayBooksListWithInvalidDateRange - Verify the behavior when an invalid date range is provided (e.g., end date is before start date) and the refresh button is clicked.",
        "8. DisplayBooksListWithBoundaryDateRange - Verify the behavior when the date range is at the boundary conditions of the available data (e.g., the earliest or latest publication date).",
        "9. DisplayBooksListWithSingleDate - Verify the behavior when the start and end dates are the same in the date range filter.",
        "10. DisplayBooksListWithFutureDateRange - Verify the behavior when the date range is in the future, beyond any book's publication date.",
        "11. DisplayBooksListWithPastDateRange - Verify the behavior when the date range is entirely in the past, before any book's publication date.",
        "12. PaginationFunctionalityAfterFiltering - Verify that pagination works correctly after filtering the books list by a date range."
    ],
    "Display the list of genres": [
        "1. DisplayInitialGenresList - Verify that the initial list of genres is displayed correctly upon selecting the \"Genres\" item.",
        "2. DisplayGenresListWithMultiplePages - Verify that genres are paginated correctly when more than 10 items are available.",
        "3. NavigateToNextPage - Verify that clicking the \"Next\" button displays the next 10 genres.",
        "4. NavigateToPreviousPage - Verify that after moving forward, clicking a \"Previous\" button (if applicable) displays the previous 10 genres.",
        "5. NavigateToSpecificPage - Verify that clicking a specific page number navigates to that page and displays the correct genres.",
        "7. DisplayEmptyGenresList - Verify the behavior when there are no genres in the database.",
        "9. RefreshGenresList - Verify that the list updates correctly after a new genre is added or an existing one is modified or deleted."
    ],
    "Edit the author": [
        "1. Successful Author Update",
        "2. Update Author with Incorrect Data",
        "3. Update Author with Empty Mandatory Fields",
        "4. Update Author with Exceedingly Long Data",
        "5. Update Author with SQL Injection or XSS Attack Vectors",
        "6. Update Author and Cancel Operation",
        "7. Update Author and Navigate Away Using Menu Before Submitting",
        "8. Update Author with Special Characters in Name",
        "9. Update Author with HTML Tags in Bio",
        "10. Update Author with Duplicate Name",
        "13. Update Author with External Links in Bio",
        "14. Update Author with Script Tags in Bio (Testing for Script Injection)"
    ],
    "Edit the book": [
        "1. Update Book with Valid Data",
        "2. Update Book with Invalid Title",
        "3. Update Book with Invalid Author",
        "4. Update Book with Invalid Genre",
        "5. Update Book with Invalid Description",
        "6. Update Book with Invalid Publication Date",
        "7. Update Book with Empty Mandatory Fields",
        "8. Update Book and Press Cancel",
        "9. Update Book and Navigate Away Using Menu Before Submitting",
        "10. Update Book with Special Characters in Fields",
        "11. Update Book with SQL Injection Attack Vectors",
        "12. Update Book with XSS Attack Vectors in Fields",
        "13. Update Book with Extremely Long Data in Each Field",
        "14. Update Book with Past Publication Date",
        "15. Update Book with Future Publication Date Beyond Reasonable Future",
        "16. Update Book with Non-Existent Author",
        "17. Update Book with Non-Existent Genre",
        "18. Update Book with Duplicate Title and Author"
    ],
    "Edit the genre": [
        "1. Successful Genre Update",
        "2. Update Genre with Invalid Data",
        "3. Update Genre with Empty Fields",
        "4. Update Genre with Duplicate Name",
        "7. Navigate Away Using Menu Before Submitting",
        "8. Navigate Away Using Menu After Data Entry",
        "9. Update Genre with Special Characters in Name",
        "10. Update Genre with Excessively Long Name",
        "11. Update Genre with HTML Tags in Description",
        "12. Update Genre with Script Injection in Description",
        "14. Update Genre with SQL Injection in Name",
        "15. Update Genre with JavaScript Code in Name"
    ],
    "Remove the author": [
        "1. Successful Author Deletion",
        "2. Cancel Author Deletion via Cancel Button",
        "3. Cancel Author Deletion via Navigation to \"Books\"",
        "4. Cancel Author Deletion via Navigation to \"Genres\"",
        "5. Cancel Author Deletion via Navigation to \"Authors\"",
        "6. Cancel Author Deletion via Navigation to \"Home\"",
        "7. Cancel Author Deletion via Navigation to \"API\"",
        "9. Delete Author with Associated Books (Constraint Check)"
    ],
    "Remove the book": [
        "1. Successful Book Deletion",
        "2. Cancel Book Deletion via Cancel Button",
        "3. Cancel Book Deletion via Navigation to \"Books\"",
        "4. Cancel Book Deletion via Navigation to \"Genres\"",
        "5. Cancel Book Deletion via Navigation to \"Authors\"",
        "6. Cancel Book Deletion via Navigation to \"Home\"",
        "7. Cancel Book Deletion via Navigation to \"API\"",
        "8. Attempt to Delete Non-Existent Book"
    ],
    "Remove the genre": [
        "1. Successful Genre Deletion",
        "2. Cancel Genre Deletion via Cancel Button",
        "3. Cancel Genre Deletion via Navigation to Books",
        "4. Cancel Genre Deletion via Navigation to Genres",
        "5. Cancel Genre Deletion via Navigation to Authors",
        "6. Cancel Genre Deletion via Navigation to Home",
        "7. Cancel Genre Deletion via Navigation to API"
    ]
}

In [79]:
# usecase_scenario = """{'Add new topic': ['1. Successful Topic Addition', '2. Adding Topic with Duplicate Name', '3. Cancel Topic Addition', '4. Adding Topic without Name', '5. Adding Topic without Description', '6. Adding Topic with Special Characters in Name', '7. Adding Topic with Long Name', '8. Adding Topic with Long Description'], 'AddLesson': ['1. Successful Lesson Addition', '2. Adding Lesson with Already Used Name', '3. Cancel Adding New Lesson', '4. Navigate to Add New Lesson Page via Navbar Option', '5. Adding Lesson Without Name', '6. Adding Lesson Without Description', '7. Adding Lesson with Special Characters in Name', '8. Adding Lesson with Long Name and Description', '9. Adding Lesson with HTML Tags in Name or Description', '10. Adding Lesson with Script Injection in Name or Description'], 'AddQuestion - Copy': ['1. Successful Question Creation', '2. Access Lesson Management from Homepage', '3. Skip Compulsory Field "Sentence"', '4. Skip Compulsory Field "Correct Answer"', '5. Skip Compulsory Field "Other Option 1"', '6. Skip Compulsory Field "Other Option 2"', '7. Skip Compulsory Field "Other Option 3"', '8. Cancel Question Creation and Choose to Stay', '9. Enter Invalid Field Input', '10. Leave Page Without Saving Question', '11. Cancel Question Creation and Confirm Cancelation'], 'AddQuestion ': ['1. Successful question creation for selection test', '2. Access Lesson Management from Homepage', '3. Skip compulsory field "sentence" and handle error', '4. Skip compulsory field "correct answer" and handle error', '5. Skip compulsory field "other option 1" and handle error', '6. Skip compulsory field "other option 2" and handle error', '7. Skip compulsory field "other option 3" and handle error', '8. Cancel question creation and choose to stay', '9. Enter invalid sentence and handle error', '10. Leave page without saving question', '11. Cancel question creation and confirm cancellation'], 'EditTest': ['1. Successful Test Edit', '2. Cancel Edit and Exit Without Saving', '5. Save Edited Test With Mandatory Fields Missing', '7. Edit Test With Special Characters in Vocabulary', '8. Edit Test With Maximum Length of Vocabulary', '9. Edit Test With Minimum Length of Vocabulary'], 'Flashcard': ['1. Successful Navigation and Learning Flow', '2. Navigation with Back Button and Stay Decision', '3. Continue Studying After Completion', '4. Direct Access to Incomplete Topic from Home Page', '5. Attempt to Navigate to Previous Flashcard on First Card', '6. Exit During Flashcard Study and Mark Topic as Incomplete'], 'Handbook': ['1. View Vocabulary List', '2. Scroll Vocabulary List', '3. Search Vocabulary Successfully', '4. Search Meaningless Vocabulary', '5. Exclude Vocabulary from Review Test', '6. Include Vocabulary in Review Test'], 'Learning days history': ['1. **View Continuous Learning Days Scenario** - This scenario covers the basic flow where a learner checks their continuous learning days without any interruptions in their learning streak.', '2. **View Learning Days After Break Scenario** - This scenario tests the functionality when a learner has recently broken their learning streak and checks their learning days, which should reset to one.'], 'ListenToPronunciation': ['1. Successful Pronunciation Test Completion', '2. Incorrect Answer Selection', '6. Attempting Test Without Being Logged In', '8. Refreshing Page During Test', '9. Closing Application During Test', '11. Skipping Pronunciation Test Question'], 'Login': ['1. Successful Login with Username', '2. Successful Login with Email', '3. Login with Wrong Password', '4. Login with Unregistered Username', '5. Login with Blank Username Field', '6. Login with Blank Password Field'], 'Registry': ['1. Successful Registration with Valid Username and Password', '2. Successful Registration with Email', '4. Registration with Invalid Password', '5. Registration with Existed Username', '7. Registration with Password Same as Username', '8. Registration with Username Below Minimum Length', '9. Registration with Username Above Maximum Length', '10. Registration with Password Below Minimum Length', '11. Registration with Password Above Maximum Length', '12. Registration with Email in Incorrect Format', '13. Registration with Non-Existent Email Address'], 'Review test': ['1. Successful Review Completion with Correct Answers', '2. Review Completion with Incorrect Answers', '3. Review Process with Mixed Correct and Incorrect Answers', '4. Exiting Review and Returning to Continue', '5. Exiting Review and Confirming Exit', '6. Review with Questions Method', '7. Review with Sound Method', '8. Review with Image Method', '9. Review with Meanings Method', '10. Last Word Review and Return to Home Page', '11. Review with No Words in Review List'], 'WordSortingTest': ['1. Successful Word Sorting with Vocabulary Definition', '2. Successful Word Sorting with Image Clue', '3. Incorrect Word Sorting with Vocabulary Definition', '4. Incorrect Word Sorting with Image Clue', '8. Word Sorting with Fewer Letters than Required']}"""
# data_str = usecase_scenario.replace("'", '"')
# usecase_scenario_json = json.loads(sanitize_string(data_str))

In [80]:
for key, value in usecase_scenario.items():
  usecase = key
  usecase_name =usecase
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  for testscenario in value:
    content = "Test scenarios:"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
    print(content)
    promptExtractCondition = [
      { "role": "system", "content": TEST_CASE_GENERATOR},
      { "role": "user", "content": content}
    ]
    gpt_response = askJSON(promptExtractCondition, client, model)
    print(gpt_response)
    print()
    json_data = json.loads(gpt_response)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
    testcase_path = os.path.join (save_path,f"{usecase_name}-{testscenario}.txt" )
    write_json_to_txt_raw(json_data,testcase_path )
  print("f{usecase_name} gen test case complete")

Test scenarios:1. Successful Topic Addition
Use case:use case id: U17
use case name: adding new topic
objective: This use case allows administrator to add a new english learning topic.
actor: administrator

basic flow:
step 1: Administrator selects "Add new topic" tab on the navigation bar.
step 2: The system redirects Administrator to the adding topic viewing page.
step 3: Administrator adds the name of the topic in the name field.
step 4: Administrator adds the description of the topic in the description field.
step 5: Administrator selects "Save" to create a new topic.

Alternative flow: Administrator adds an already used name for the topic.
At step 3 of the basic flow: Administrator adds a name that is already used in the existed topics in the name field.
step 4: Administrator adds the description of the topic in the description field.
step 5: Administrator selects "Save" to create a new topic.
step 6: The system shows notification that the name is already used, prompting Administr